# LAB 4 : Data cleansing

| Member Name                         | Student ID           |
|----------------------------------------|-----------------------|
|Kanyaluck    Chimchome            |    64070501003   |
|Boonyarit     Samran                 |    64070501028   |  	
|Warisara      Patib                     |   64070501044    |  	
|Chanidapa   Chanama                |  64070501090    |	
|Nontawat      Kunlayawuttipong  |  64070501093   	|

## importing libraries and content files

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cd_card = pd.read_csv("credit_card_transaction/credit_card_info.csv")
cd_tst = pd.read_csv("credit_card_transaction/credit_transaction.csv")
cd_user = pd.read_csv("credit_card_transaction/credit_users.csv")
mc_code = pd.read_csv("credit_card_transaction/mcc_codes.csv")

## viewing datatype and missing data observation

### credit card

In [ ]:
cd_card.head()

In [ ]:
cd_card.info(memory_usage="deep")

### credit transaction

In [ ]:
cd_tst.head()

In [ ]:
cd_tst.info()

In [ ]:
cd_tst.isna().sum()

### credit users

In [ ]:
cd_user.head()

In [ ]:
cd_user.info()

In [ ]:
cd_user.isna().sum()

### mcc_codes

In [ ]:
mc_code.head()

In [ ]:
mc_code.info()

In [ ]:
mc_code.isna().sum()

## Coding 

In [ ]:
user_prof = pd.DataFrame()

In [ ]:
user_prof["user_id"] = cd_user.user_id

1. Ages in year

In [ ]:
# get referrence date from max transaction date
ref_date = max(pd.to_datetime(cd_tst.transaction_datetime))
ref_date

In [ ]:
user_prof["current_age"] = (ref_date - pd.to_datetime(cd_user.date_of_birth)).dt.days // 365

2. Number of active and expired cards

In [ ]:
from pandas.tseries.offsets import MonthEnd

# pd.to_datetime(cd_card["expires"]).dt.strftime('%Y-%m')
tempDate = pd.to_datetime(cd_card["expires"], format="%m/%Y")
tempExp = pd.DataFrame((tempDate - ref_date).dt.days)
tempExp["user_id"] = cd_card["user_id"]
tempExp["expired_cards"] = tempExp.expires < 0
test = tempExp.copy()
tempExp = tempExp.groupby(["user_id","expired_cards"])['expires'].agg(['count']).unstack(fill_value=0)

user_prof = pd.merge(user_prof, tempExp, on="user_id")
user_prof.columns = ['user_id', 'current_age', 'active_cards', 'expired_cards']

In [ ]:
user_prof

3. Total credit limit of active cards

In [ ]:
# Credit_limit_active
tempActiveCard = pd.DataFrame((tempDate - ref_date).dt.days)
tempActiveCard["user_id"] = cd_card['user_id']
tempActiveCard["active_cards"] = tempActiveCard.expires > 0

# Copy Dataframe
tempCl = tempActiveCard.copy()

# Add Dataframe
tempCl["credit_limit_active"] = cd_card['credit_limit']
tempCl['credit_limit_active'] = tempCl.credit_limit_active.str.extract('(\d+)').astype(int)

# Drop expired card
tempCl = tempCl[tempCl.active_cards!=False]

# Each user_id have 2 type of cards -> active / expired
# and different amount of them
tempCl = tempCl.groupby(["user_id"])['credit_limit_active'].agg(['sum']).unstack(fill_value=0).reset_index(name='credit_limit_active')

# Merge in to user_prof
user_prof = pd.merge(user_prof,tempCl, on = "user_id")
user_prof = user_prof.drop(columns='level_0')

4. Time since first card openned (acct_open_date) in months

In [ ]:
# Create Dataframe to keep user_id and acct_open_date from credit_card_info
tempOC = pd.DataFrame()
tempOC["user_id"] = cd_card['user_id']
tempOC["time_since_first_card_months"] = pd.to_datetime(cd_card['acct_open_date'])

# Select min_date_time of acct_open_date in each user_id
tempOC = tempOC.groupby(["user_id"])['time_since_first_card_months'].min().reset_index()

# ref_date - min_date_time in each user_id
# and turn value in to Month
tempOC["time_since_first_card_months"] = (ref_date - tempOC['time_since_first_card_months']) // np.timedelta64(1, 'M')

# Merge in to user_prof
user_prof = pd.merge(user_prof,tempOC, on = "user_id")

5. Time since last transaction in days

In [ ]:
# Create Dataframe
tempLast = pd.DataFrame()
tempLast["user_id"] = cd_tst['user_id']
tempLast["time_since_last_transaction_days"] = pd.to_datetime(cd_tst['transaction_datetime'])

# Select max_date_time of last transaction day in each user_id
tempLast = tempLast.groupby(["user_id"])['time_since_last_transaction_days'].max().reset_index()

# ref_date - max_date_time in each user_id
# and turn value in to Day
tempLast['time_since_last_transaction_days'] = (ref_date - tempLast['time_since_last_transaction_days']) // np.timedelta64(1, 'D')

# Merge in to user_prof
user_prof = pd.merge(user_prof,tempLast, on = "user_id")

6. Distribution of active card type (Debit, Credit, Prepaid)

In [ ]:
# Copy Dataframe
temptypecard = tempActiveCard.copy()
temptypecard["card_type"] = cd_card['card_type']

# Drop expired card
temptypecard = temptypecard[temptypecard.active_cards!=False]
testtypecard = pd.pivot_table(temptypecard,index=['user_id'],columns=['card_type'],values=['active_cards'],aggfunc='count').fillna(0).reset_index()

# Merge in to user_prof
user_prof = pd.merge(user_prof,testtypecard, on = "user_id")

# Rename columns
# Credit             ->     active_card_credit
# Debit              ->     active_card_debit
# Debit (Prepaid)    ->     active_card_debit_(prepaid)
user_prof = user_prof.rename(columns={('active_cards', 'Credit'):"active_card_credit",('active_cards', 'Debit'):"active_card_debit",('active_cards', 'Debit (Prepaid)'):"active_card_debit_(prepaid)"})

7. Number of merchants that this user made transaction

In [ ]:
# Create Dataframe to keep user_id and merchant_id from credit_transaction
tempM = pd.DataFrame()
tempM["user_id"] = cd_tst['user_id']
tempM["num_merchants"] = cd_tst['merchant_id']

# Each user_id have different number of merchants
tempM = tempM.groupby(["user_id"]).agg({"num_merchants":"nunique"})

# Merge in to user_prof
user_prof = pd.merge(user_prof,tempM, on = "user_id")

8. Number of states that this user made transaction

In [ ]:
# Create Dataframe to keep user_id and merchant_state from credit_transaction
tempstates = pd.DataFrame()
tempstates["user_id"] = cd_tst['user_id']
tempstates["num_states"] = cd_tst['merchant_state']

# Each user_id have different number of states
tempstates = tempstates.groupby(["user_id"]).agg({"num_states":"nunique"})

# Merge in to user_prof
user_prof = pd.merge(user_prof,tempstates, on = "user_id")

9. Number of times that user made transaction outside the USA (online transaction counts as nonUSA transaction)

In [ ]:
# Create Dataframe to keep user_id and merchant_city from credit_transaction
tempnonUSA = pd.DataFrame()
tempnonUSA["user_id"] = cd_tst['user_id']
tempnonUSA["abroad_tx"] = cd_tst['merchant_city']

# Check if value in abroad_tx is "ONLINE" then return True
#       if not return False
tempnonUSA['abroad_tx'] = tempnonUSA['abroad_tx'].isin(['ONLINE'])

# Then keep row that return True
tempnonUSA = tempnonUSA[tempnonUSA.abroad_tx == True]

# Each user_id have different nonUSA transaction times
tempnonUSA = tempnonUSA.groupby(["user_id"]).agg({"abroad_tx":"size"}).reset_index()

# Merge in to user_prof
user_prof = pd.merge(user_prof,tempnonUSA, on = "user_id",how = "left")

In [ ]:
user_prof.head(10)